In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [8]:
park = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/천안시 공원정보/충청남도_천안시_도시공원정보_20230711.csv',encoding='cp949')
park.head(2)
park_cp = park[['관리번호','공원명','위도','경도']]
park_cp['location'] = pd.Series(zip(park_cp['위도'],park_cp['경도']))
park_cp.head(1)


,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자
0,44131-00043,용곡공원(용곡4),어린이공원,NaN,충청남도 천안시 동남구 용곡동 597,36.787417,127.135552,2300.0,NaN,조합놀이대,NaN,NaN,NaN,1993-02-18,충청남도 천안시청,041-521-2772,2023-07-11
1,44131-00044,눈들공원(용곡1),어린이공원,NaN,충청남도 천안시 동남구 용곡동 601,36.792611,127.139216,1500.0,NaN,조합놀이대+흔들놀이,NaN,NaN,NaN,1987-06-12,충청남도 천안시청,041-521-2772,2023-07-11


,관리번호,공원명,위도,경도,location
0,44131-00043,용곡공원(용곡4),36.787417,127.135552,"(36.78741709, 127.1355518)"


In [6]:
bus = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.pickle')
bus_cp = bus[['정류장번호','정류장명','위도','경도','coord2address','주변시설의총합']]
bus_cp.reset_index(drop=True, inplace=True)

bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

,정류장번호,정류장명,위도,경도,coord2address,주변시설의총합,location
0,ASB285000077,성환8리,36.921468,127.122774,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)"
1,ASB285000078,송덕,36.923783,127.118976,"{'road_address': None, 'address': {'address_na...",515,"(36.92378302, 127.11897568)"
2,ASB285000079,국립축산과학원,36.925931,127.111136,"{'road_address': None, 'address': {'address_na...",245,"(36.92593139, 127.11113568)"
3,ASB285000094,성환8리,36.921479,127.122505,"{'road_address': None, 'address': {'address_na...",539,"(36.9214794, 127.1225049)"
4,ASB285000095,성환터미널,36.917221,127.132732,"{'road_address': None, 'address': {'address_na...",620,"(36.91722136, 127.13273235)"


In [10]:
park_cp2 = park_cp.copy()
bus_cp2 = bus_cp.copy()

In [11]:
# x : 경도, y : 위도
# data['distance'] = data.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']).km, axis=1)
def calculate_distance(bus_cp, company_house):
    import geopy.distance

    data_list = []

    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],'location':bus_cp.at[idx,'location'],
            '공원명':company_house.at[jdx,'공원명'],'공원좌표':company_house.at[jdx,'location'],'공원관리번호':company_house.at[jdx,'관리번호'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    
# company_house.reset_index(drop=True,inplace=True)
bus_Around_company_house = calculate_distance(bus_cp2,park_cp2)

100%|██████████| 4324/4324 [01:51<00:00, 38.93it/s]


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


In [14]:
bus_park_distance = bus_Around_company_house.copy()
bus_park_distance

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


In [17]:
# bus_park_distance.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시 공원정보/버스정류장과공원사이의거리.pickle')

In [22]:
bus_park_distance.loc[bus_park_distance['distance']<=0.2,'정류장번호'].value_counts()

CAB285000834    2
CAB285000896    2
CAB285000940    2
CAB285000941    2
CAB285000942    2
               ..
CAB285000917    1
CAB285000918    1
CAB285000919    1
CAB285000920    1
GGB285500030    1
Name: 정류장번호, Length: 338, dtype: int64